In [1]:
import os

In [2]:
! pip install -U transformers adapters transformers[torch] torch

zsh:1: no matches found: transformers[torch]


In [3]:
TRANSFORMERS_CACHE="checkpoints/hf_model"
HF_DATASETS_CACHE="checkpoints/hf_model"
HF_METRICS_CACHE="checkpoints/hf_model"

cache_dir=TRANSFORMERS_CACHE

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
import torch
import argparse
import os

from collections import OrderedDict

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
    AutoConfig,
)

/Users/divyajyotipanda/miniconda3/envs/nlpproj/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [6]:
from adapters import AutoAdapterModel

/Users/divyajyotipanda/miniconda3/envs/nlpproj/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/divyajyotipanda/miniconda3/envs/nlpproj/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [7]:
ADAPTER_SAVE_PATH = 'Merged_Adapters_2'
# TASK_NAME = "EXTROVERT_INTROVERT"

In [8]:
label2id = {
    "Strongly Disagree": 0,
    "Disagree": 1,
    "Slightly Disagree": 2,
    "Neutral": 3,
    "Slightly Agree": 4,
    "Agree": 5,
    "Strongly Agree": 6
}

id2label = {
    0: "Strongly Disagree",
    1: "Disagree",
    2: "Slightly Disagree",
    3: "Neutral",
    4: "Slightly Agree",
    5: "Agree",
    6: "Strongly Agree"

}

In [9]:
def merge_op(_lambda = [0.25]*4):
    #assert 0 <= _lambda <= 1, "lambda out of bounds"
    # for l in _lambda:
    #     assert 0 <= l <= 1, "lambda out of bounds"
    model_name_or_path=["bert-base-uncased"] * 4
    task_name=["EXTROVERSION", "SENSOR", "THINKER", "JUDGING"] # ["cola","sst2"]
    #pretrained_adapter=["lingaccept/cola@ukp", "lingaccept/cola@ukp"]#["lingaccept/cola@ukp","sentiment/sst-2@ukp"]
    # ! TODO: try to run on multiple
    #pretrained_adapter = ["lora/FEELER/best_model_dir", "lora/PERCEIVER/best_model_dir"]
    #pretrained_adapter = ["new_ia3/" + x + "/best_model_dir" for x in task_name]
    pretrained_adapter = [x + "/best_model_dir" for x in task_name]
    adapter_config="ia3"
    #save_path="new_ia3/ENTJ"
    save_path="ESTJ" + "_".join([str(x) for x in _lambda])
    # ADAPTER_SAVE_PATH = '/NLPProject/'
    # TASK_NAME = "PERCEIVER"
    #coefficient_sets=[_lambda, 1 - _lambda]
    coefficient_sets = _lambda
    adapter_merge(model_name_or_path, task_name, pretrained_adapter, adapter_config, save_path, coefficient_sets = coefficient_sets, full_merge=False)

In [10]:
def adapter_merge(
    model_name_or_path,
    task_name_list,
    adapter_list,
    adapter_config,
    save_path,
    overwrite_existed = True,
    coefficient_sets = None,
    dropout_list = None,
    merge_way = "simple",
    merge_head = False,
    full_merge:bool = False,
    fisher_list = [],
    fisher_floor = 1e-6,
    fisher_favor_target_model = True,
    fisher_normalization = True,
    *args,
    **kwargs):

    # what's the classification head belongs to

    # merged_name=task_name_list
    # #merged_name.sort()
    # merged_name="_".join(merged_name)
    merged_name = save_path.split('/')[-1]
    # check if already existed
    will_save_path=os.path.join(save_path, merged_name + "_".join([str(x) for x in coefficient_sets]))
    if os.path.isdir(will_save_path):
        if(overwrite_existed):
            print("Overwrite the existed adapter.")
        else:
            print("Merged adapter already created.")
            return

    if full_merge:
        range_list=model_name_or_path
    else:
        model_name_or_path=model_name_or_path[0]
        range_list=adapter_list
    # model-level
    if coefficient_sets == None:
        coefficient_sets = len(range_list) * [ 1 / len(range_list)]
    else:
        # suppose that len(coefficient_sets) == len(adapter_list)
        # TODO: assert len(coefficient_sets) == len(adapter_list)
        coefficient_sets_orig = coefficient_sets.copy()
        coefficient_sets = [float(x) for x in coefficient_sets]
        #coefficient_sets = [ x/sum(coefficient_sets) for x in coefficient_sets]
        coefficient_sets = [ x/10 for x in coefficient_sets]
    print(coefficient_sets)

    state_dict={}
    merged_dict={}
    merged_keys = None
    merged_values=[]
    sum_factors = None

    for i, _ in enumerate(range_list):
        if full_merge:
            model_name_or_path=range_list[i]
        else:
            adapter=range_list[i]
        # have to load model every time when we load adapter
        if dropout_list == None:
            dropout=0.1
        else:
            dropout=float(dropout_list[i])
        config = AutoConfig.from_pretrained(
            model_name_or_path,
            attention_probs_dropout_prob=dropout,
            hidden_dropout_prob=dropout
        )
        tokenizer = AutoTokenizer.from_pretrained(
            model_name_or_path,
            cache_dir=cache_dir,
        )
        # model = AutoModelForSequenceClassification
        # model = AutoModelForSeq2SeqLM
        model = AutoAdapterModel.from_pretrained(
            model_name_or_path,
            cache_dir=cache_dir,
            config=config,
        )
        model.resize_token_embeddings(len(tokenizer))

        #print(adapter)
        #print(model)
        if full_merge == False:
            model.load_adapter(
                adapter,
                config=adapter_config,
                load_as=merged_name,
            )

        state_dict = model.state_dict()
        if dropout == 0:
            head_keys = [hk for hk in state_dict.keys() if "head" in hk]
            for hk in head_keys:
                if '0' in hk:
                    hk_n = '1'.join(hk.split('0'))
                elif '2' in hk:
                    hk_n = '4'.join(hk.split('2'))
                state_dict[hk_n]=state_dict[hk]
                state_dict.pop(hk)
        # breakpoint()

        if merged_keys == None:
            if full_merge:
                if merge_head:
                    merged_keys = [mk for mk in state_dict.keys()]
                else:
                    merged_keys = [mk for mk in state_dict.keys() if ("head" not in mk) and ("classifier" not in mk)]
            else:
                merged_keys = [mk for mk in state_dict.keys() if  ("adapters" in mk) or ("ia3" in mk)] # adapters / lora
                head_keys = [hk for hk in state_dict.keys() if "head" in hk]
                if merge_head:
                    merged_keys = merged_keys + head_keys
            merged_values = [0] * len(merged_keys)
            sum_factors = [0] * len(merged_keys)

        # parameter-level
        factors = None
        if merge_way == "fisher":
            fisher = torch.load(fisher_list[i])
            fisher = {k:v.detach() for k,v in fisher.items()}

            # normalization
            if fisher_normalization:
                vs=[v for v in fisher.values()]
                norm_constants=torch.sqrt(sum([torch.sum(torch.square(v)) for v in fisher.values()]))
                coefficient_sets[i]=coefficient_sets[i]/norm_constants

            # fisher_floor: avoid numerical problem
            if (not fisher_favor_target_model) or (i == len(range_list)-1):
                fisher = {k:torch.maximum(v, torch.tensor(fisher_floor)) for k,v in fisher.items()}

            # TODO:assert set(merged_keys)==set(fisher.keys())
            factors = OrderedDict([(mk, fisher[task_name_list[i].join(mk.split(merged_name))]*coefficient_sets[i]) for mk in merged_keys])
        elif merge_way == "simple":
            factors = OrderedDict(zip(merged_keys, [coefficient_sets[i]]*len(merged_keys)))

        sum_factors = [sf+f for sf,f in zip(sum_factors, factors.values())]

        # Add together
        new_values = [state_dict[mk]*factors[mk] for mk in merged_keys]
        merged_values = [mv+nv for mv,nv in zip(merged_values, new_values)]

        if i == len(range_list)-1:
            merged_values = [torch.div(mv, sf) for mv,sf in zip(merged_values, sum_factors)]
            merged_dict = dict(zip(merged_keys, merged_values))

            state_dict.update(merged_dict)
            if dropout == 0:
                config = AutoConfig.from_pretrained(
                    model_name_or_path,
                )
                model = AutoAdapterModel.from_pretrained(
                    model_name_or_path,
                    cache_dir=cache_dir,
                    config=config,
                )
                if full_merge == False:
                    model.load_adapter(
                        adapter,
                        config=adapter_config,
                        load_as=merged_name,
                    )
            model.load_state_dict(state_dict)
            if full_merge:
                config = AutoConfig.from_pretrained(
                    model_name_or_path,
                    # num_labels=num_labels,
                    finetuning_task=task_name_list[i],
                    cache_dir=cache_dir,
                )
                tokenizer = AutoTokenizer.from_pretrained(
                    model_name_or_path,
                    cache_dir=cache_dir,
                )
                print("Save Path full_merge")
                model.save_pretrained(save_path + "_".join([str(x) for x in coefficient_sets_orig]))
                config.save_pretrained(save_path + "_".join([str(x) for x in coefficient_sets_orig]))
                tokenizer.save_pretrained(save_path + "_".join([str(x) for x in coefficient_sets_orig]))
            else:
                print("Save Path")
                model.save_all_adapters(save_path)


In [11]:
# # hyperparam sweep over the lambda

# import numpy as np

# lmbda_vals = np.linspace(0.0, 1.0, num=10)

# sweep_results = []

# for lmda in lmbda_vals:
#     # make merge_op return path of saved merge adapter
#     merge_op(lmda)
#     # load checkpoint
    
#     results = eval(choices, task_name=)
#     sweep_results.append({
#         'lambda' : lmda,
#         'result' : results
#     }) 

#     print(lmda, results)

In [12]:
# for l1 in range(1, 10):
#     for l2 in range(1, 10 - l1):
#         for l3 in range(1, 10 - l1 - l2):
#             #print([l1 / 20, l2 / 20, l3 / 20, (20 - l1 - l2 - l3) / 20])
#             merge_op([l1, l2, l3, 10 - l1 - l2 - l3])
            

[0.1, 0.1, 0.1, 0.7]


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream t

Save Path
[0.1, 0.1, 0.2, 0.6]


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream t

Save Path
[0.1, 0.1, 0.3, 0.5]


...

Save Path
[0.1, 0.4, 0.4, 0.1]


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream t

Save Path
[0.1, 0.5, 0.1, 0.3]


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [17]:
def merge_op_wrapper(_lambda = [0.25]*4):
    traits = [[("EXTROVERSION", "E"), ("INTROVERSION", "I")], [("SENSOR", "S"), ("INTUITIVE", "N")], [("THINKER", "T"), ("FEELER", "F")], [("JUDGING", "J"), ("PERCEIVER", "P")]]
    for tr1 in traits[0]:
        for tr2 in traits[1]:
            for tr3 in traits[2]:
                for tr4 in traits[3]:
                    task_name = [x[0] for x in [tr1, tr2, tr3, tr4]]
                    personality = "".join([x[1] for x in [tr1, tr2, tr3, tr4]])
                    model_name_or_path=["bert-base-uncased"] * 4
                    pretrained_adapter = [x + "/best_model_dir" for x in task_name]
                    adapter_config="ia3"
                    save_path=personality + "_".join([str(x) for x in _lambda])
                    coefficient_sets = _lambda
                    adapter_merge(model_name_or_path, task_name, pretrained_adapter, adapter_config, save_path, coefficient_sets = coefficient_sets, full_merge=False)

In [19]:
for l1 in range(1, 10):
    for l2 in range(1, 10 - l1):
        for l3 in range(1, 10 - l1 - l2):
            #print([l1 / 20, l2 / 20, l3 / 20, (20 - l1 - l2 - l3) / 20])
            print([l1, l2, l3, 10 - l1 - l2 - l3])
            merge_op_wrapper([l1, l2, l3, 10 - l1 - l2 - l3])
            print()
            print()
            print()

[1, 1, 1, 7]
[0.1, 0.1, 0.1, 0.7]


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream t

Save Path
[0.1, 0.1, 0.1, 0.7]


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream t

Save Path
[0.1, 0.1, 0.1, 0.7]


...

Save Path
[0.7, 0.1, 0.1, 0.1]


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream t

Save Path
[0.7, 0.1, 0.1, 0.1]


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream t

Save Path



